In [1]:
import pandas as pd
import numpy as np
import json

import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.diagnostic import linear_rainbow, het_breuschpagan
from statsmodels.stats.outliers_influence import variance_inflation_factor

from sklearn.preprocessing import LabelEncoder

In [2]:
df = pd.read_json('../Data/combined_with_dummies.json')
df.head()

,DocumentDate,SalePrice,Area,SqFtLot,WfntFootage,SqFtTotLiving,SqFtDeck,SqFtOpenPorch,cdtn_2.0,cdtn_3.0,...,wtrloc_4.0,wtrloc_5.0,wtrloc_6.0,wtrloc_7.0,wtrloc_8.0,wtrloc_9.0,wtrpoorq_1.0,wtrresacc_1.0,wtrresacc_2.0,wtrresacc_3.0
8576000260,1546300800000,583950,57,5944,0,3270,0,250,0,1,...,0,0,0,0,0,0,0,0,0,0
7691000060,1546300800000,539950,41,8389,0,3200,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
53300380,1546300800000,642950,51,4000,0,2487,0,45,0,1,...,0,0,0,0,0,0,0,0,0,0
2525059126,1546300800000,541000,91,13503,0,1030,0,210,0,0,...,0,0,0,0,0,0,0,0,0,0
53300290,1546300800000,647950,51,5362,0,2487,0,45,0,1,...,0,0,0,0,0,0,0,0,0,0


In [3]:
#pd.plotting.scatter_matrix(df, figsize =[12,12]); way to big baby

In [4]:
df.corr()

,DocumentDate,SalePrice,Area,SqFtLot,WfntFootage,SqFtTotLiving,SqFtDeck,SqFtOpenPorch,cdtn_2.0,cdtn_3.0,...,wtrloc_4.0,wtrloc_5.0,wtrloc_6.0,wtrloc_7.0,wtrloc_8.0,wtrloc_9.0,wtrpoorq_1.0,wtrresacc_1.0,wtrresacc_2.0,wtrresacc_3.0
DocumentDate,1.000000,0.044813,-0.014539,0.010500,0.005998,0.002973,-0.004144,0.001581,0.010762,0.014299,...,0.000121,-0.003610,-0.000207,0.008194,-0.000857,-0.003721,0.005124,0.005569,-0.002088,-0.004457
SalePrice,0.044813,1.000000,-0.040850,0.015736,0.076848,0.433613,0.135837,0.233021,-0.024300,0.069641,...,0.001365,0.014955,0.307676,0.106660,0.015542,0.010445,0.001485,0.017192,0.034847,0.003806
Area,-0.014539,-0.040850,1.000000,0.048688,0.026620,0.105163,-0.015526,0.120239,-0.017563,0.010680,...,-0.007962,-0.012671,-0.018387,-0.001613,0.019630,0.045555,0.008409,0.025600,0.030173,0.010136
SqFtLot,0.010500,0.015736,0.048688,1.000000,0.786419,0.034830,0.029570,0.046943,0.008638,-0.017324,...,-0.000450,-0.001148,0.001797,0.000145,0.005812,0.141863,0.007512,-0.000361,0.009323,0.009860
WfntFootage,0.005998,0.076848,0.026620,0.786419,1.000000,0.029502,0.039544,0.034453,0.001001,-0.013737,...,0.007135,0.020731,0.112736,0.072021,0.120769,0.389070,0.085995,0.042915,0.117965,0.107156
SqFtTotLiving,0.002973,0.433613,0.105163,0.034830,0.029502,1.000000,0.214127,0.438153,-0.064036,0.150484,...,-0.002840,-0.004145,0.102113,0.049816,0.034416,0.009731,0.006395,-0.005266,0.020284,0.007292
SqFtDeck,-0.004144,0.135837,-0.015526,0.029570,0.039544,0.214127,1.000000,-0.037964,-0.025595,-0.083835,...,-0.001107,0.050491,0.067142,0.055043,0.065637,0.041546,0.016123,0.051681,0.071214,0.005005
SqFtOpenPorch,0.001581,0.233021,0.120239,0.046943,0.034453,0.438153,-0.037964,1.000000,-0.015032,0.121311,...,-0.003658,-0.001584,0.062897,0.020975,0.037316,0.039024,0.006002,-0.001407,0.004015,-0.004581
cdtn_2.0,0.010762,-0.024300,-0.017563,0.008638,0.001001,-0.064036,-0.025595,-0.015032,1.000000,-0.112483,...,-0.000512,-0.001354,0.001215,0.008311,0.006914,0.003510,0.014520,-0.002509,-0.003834,0.017598
cdtn_3.0,0.014299,0.069641,0.010680,-0.017324,-0.013737,0.150484,-0.083835,0.121311,-0.112483,1.000000,...,0.004550,0.007238,-0.001285,-0.007382,-0.020113,-0.012704,-0.006398,-0.003641,-0.013495,-0.006742


In [5]:
data_pred = df.iloc[:,2:50]
data_pred.head()

,Area,SqFtLot,WfntFootage,SqFtTotLiving,SqFtDeck,SqFtOpenPorch,cdtn_2.0,cdtn_3.0,cdtn_4.0,cdtn_5.0,...,wtrloc_4.0,wtrloc_5.0,wtrloc_6.0,wtrloc_7.0,wtrloc_8.0,wtrloc_9.0,wtrpoorq_1.0,wtrresacc_1.0,wtrresacc_2.0,wtrresacc_3.0
8576000260,57,5944,0,3270,0,250,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
7691000060,41,8389,0,3200,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
53300380,51,4000,0,2487,0,45,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2525059126,91,13503,0,1030,0,210,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
53300290,51,5362,0,2487,0,45,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
df=data_pred.corr().abs().stack().reset_index().sort_values(0, ascending=False)


df['pairs'] = list(zip(df.level_0, df.level_1))


df.set_index(['pairs'], inplace = True)


df.drop(columns=['level_1', 'level_0'], inplace = True)

df.columns = ['cc']

df.drop_duplicates(inplace=True)

In [7]:
df[(df.cc>.75) & (df.cc <1)]

,cc
pairs,
"(prptytyp_3, prptytyp_11)",0.879641
"(wtrloc_7.0, laksam_4.0)",0.857897
"(SqFtLot, WfntFootage)",0.786419
"(cdtn_3.0, cdtn_4.0)",0.758555
"(wtrloc_6.0, lakewas_4.0)",0.755890


In [8]:
data_pred.drop(columns=['prptytyp_3','prptytyp_11', 'wtrloc_7.0', 'laksam_4.0', 'cdtn_3.0', 'cdtn_4.0', 'wtrloc_6.0', 'lakewas_4.0'], inplace = True)
data_pred.shape

data_pred.drop(columns ='WfntFootage', inplace = True) #choosing to keep SqFtLot isntead of WfntFootage 
                                                        #because all house types have lots as opposed to 
                                                        #having waterfronts
data_pred.shape

(26955, 38)

In [9]:
data_pred

,Area,SqFtLot,SqFtTotLiving,SqFtDeck,SqFtOpenPorch,cdtn_2.0,cdtn_5.0,laksam_1.0,laksam_2.0,laksam_3.0,...,wtrloc_1.0,wtrloc_3.0,wtrloc_4.0,wtrloc_5.0,wtrloc_8.0,wtrloc_9.0,wtrpoorq_1.0,wtrresacc_1.0,wtrresacc_2.0,wtrresacc_3.0
8576000260,57,5944,3270,0,250,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7691000060,41,8389,3200,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
53300380,51,4000,2487,0,45,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2525059126,91,13503,1030,0,210,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
53300290,51,5362,2487,0,45,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1426059089,36,65496,3680,1000,50,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7128300190,15,3000,1280,0,190,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3331500845,81,5150,570,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
945000570,6,4650,2820,220,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
data_pred.corr()

,Area,SqFtLot,SqFtTotLiving,SqFtDeck,SqFtOpenPorch,cdtn_2.0,cdtn_5.0,laksam_1.0,laksam_2.0,laksam_3.0,...,wtrloc_1.0,wtrloc_3.0,wtrloc_4.0,wtrloc_5.0,wtrloc_8.0,wtrloc_9.0,wtrpoorq_1.0,wtrresacc_1.0,wtrresacc_2.0,wtrresacc_3.0
Area,1.000000,0.048688,0.105163,-0.015526,0.120239,-0.017563,-0.035158,0.012945,0.005455,0.008540,...,-0.009472,0.048950,-0.007962,-0.012671,0.019630,0.045555,0.008409,0.025600,0.030173,0.010136
SqFtLot,0.048688,1.000000,0.034830,0.029570,0.046943,0.008638,-0.003234,0.003223,0.000300,0.000034,...,0.000670,0.004059,-0.000450,-0.001148,0.005812,0.141863,0.007512,-0.000361,0.009323,0.009860
SqFtTotLiving,0.105163,0.034830,1.000000,0.214127,0.438153,-0.064036,-0.066231,0.047882,0.044693,0.043502,...,-0.014129,-0.002196,-0.002840,-0.004145,0.034416,0.009731,0.006395,-0.005266,0.020284,0.007292
SqFtDeck,-0.015526,0.029570,0.214127,1.000000,-0.037964,-0.025595,0.041634,0.032218,0.031692,0.043233,...,0.000919,0.077311,-0.001107,0.050491,0.065637,0.041546,0.016123,0.051681,0.071214,0.005005
SqFtOpenPorch,0.120239,0.046943,0.438153,-0.037964,1.000000,-0.015032,-0.057592,0.039264,0.005661,0.009206,...,-0.003314,0.002843,-0.003658,-0.001584,0.037316,0.039024,0.006002,-0.001407,0.004015,-0.004581
cdtn_2.0,-0.017563,0.008638,-0.064036,-0.025595,-0.015032,1.000000,-0.029036,0.007145,-0.003199,-0.002661,...,-0.000887,0.002375,-0.000512,-0.001354,0.006914,0.003510,0.014520,-0.002509,-0.003834,0.017598
cdtn_5.0,-0.035158,-0.003234,-0.066231,0.041634,-0.057592,-0.029036,1.000000,-0.002918,-0.009991,-0.007142,...,-0.003645,-0.002701,-0.002105,-0.005569,0.012145,0.000792,0.008937,-0.006285,0.002711,-0.002602
laksam_1.0,0.012945,0.003223,0.047882,0.032218,0.039264,0.007145,-0.002918,1.000000,-0.001591,-0.001323,...,-0.000441,-0.002512,-0.000255,-0.000674,-0.002921,-0.002294,-0.001110,-0.001248,-0.001907,-0.000953
laksam_2.0,0.005455,0.000300,0.044693,0.031692,0.005661,-0.003199,-0.009991,-0.001591,1.000000,-0.001205,...,-0.000402,-0.002288,-0.000232,-0.000613,-0.002660,-0.002090,-0.001011,-0.001136,0.019699,-0.000868
laksam_3.0,0.008540,0.000034,0.043502,0.043233,0.009206,-0.002661,-0.007142,-0.001323,-0.001205,1.000000,...,-0.000334,-0.001903,-0.000193,-0.000510,-0.002213,-0.001738,-0.000841,-0.000945,0.050069,-0.000722
